In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Research_Clifford/Datasets')

In [ ]:
pip install spectral

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import scipy.io as sio
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.metrics import confusion_matrix, cohen_kappa_score, accuracy_score
from spectral import *
import time

In [ ]:
# Load hyperspectral image data
data = np.load('indianpines_data.npy')
data_gt = np.load('indianpines_gt.npy')

print(data.shape)
print(data_gt.shape)

# Count number of unique classes in ground truth labels
num_classes = len(np.unique(data_gt))

# Count number of samples in each class
class_counts = np.bincount(data_gt.ravel())

# Print number of classes
print(f"Number of classes in Indian Pines: {num_classes}")

# Count number of samples in each class
#class_counts = np.bincount(data_gt.ravel())

# Print number of samples in each class
#for i, count in enumerate(class_counts):
#    print(f"Class {i+1}: {count}")

In [ ]:
#Data to be put in 2D

# Flatten data into 2D array
n_rows, n_cols, n_bands = data.shape
data_2d = data.reshape((n_rows*n_cols, n_bands))
print(data_2d.shape)

In [ ]:
# Run k-means with K-means++ initialization on HSI data, extract the clusters centers and the pixel indices in each cluster 
import numpy as np
from sklearn.cluster import KMeans

def kmeans_endmember_extraction(X, K):
    """
    Perform hyperspectral endmember extraction using k-means algorithm.
    
    Args:
    - X: 2D array of shape (N, M), where N is the number of pixels and M is the number of spectral bands.
    - K: integer, the number of endmembers to extract.
    
    Returns:
    - E: 2D array of shape (K, M), the extracted endmembers.
    """
    # Normalize the data to have zero mean and unit variance
    X_norm = (X - np.mean(X, axis=0)) / np.std(X, axis=0)
    
    # Perform k-means clustering
    kmeans = KMeans(n_clusters=K, init='k-means++', n_init=10, max_iter=300, random_state=0)
    kmeans.fit(X_norm)
    
    # Extract endmembers as the cluster centers
    E = kmeans.cluster_centers_

    lables=kmeans.fit_predict(X, y=None, sample_weight=None)

    unique_lables=np.unique(lables)

    return lables, E, unique_lables

    #return E


In [ ]:
Lables, Centers, Unique_Lables=kmeans_endmember_extraction(data_2d,16)

In [ ]:
print(Lables)

In [ ]:
print(Unique_Lables)
print(Unique_Lables.shape)
print(Unique_Lables[3])

In [ ]:
print(Centers)

In [ ]:
# Count number of samples in each class
cluster_counts = np.bincount(Lables.ravel())

In [ ]:
print(cluster_counts)

In [ ]:
# Select the pixel of interest
#x, y = 10, 20

# Extract the spectral signature of the pixel
pixel_spectrum = Centers[0,:]

# Plot the spectral signature
plt.plot(pixel_spectrum)
plt.xlabel('Band Index')
plt.ylabel('Intensity')
#plt.title('Spectral Signature of Pixel at ({}, {})'.format(x, y))
plt.show()


In [ ]:
# For each cluster, find the closest pixel to the center of the cluster

# For each pixel if its lable is '3' it has to go to the cluster with index '3'

# for i in Lables:
#   if Lables[i]==0:
#     Clust_Dict={}
#     Clust_Dict["#0"]=data_2d[i,:]

# print(Clust_Dict)

Clust_Dict={}
j=0
for i in data_2d:
  k=Lables[j]
  #print(Clust_Dict)
  if(str(k) not in Clust_Dict.keys()):
    Clust_Dict[str(k)]=i
    Clust_Dict[str(k)]=list(Clust_Dict[str(k)])
    # print(Clust_Dict[str(k)])
    # break
  else:
    Clust_Dict[str(k)].append(i)
    #Clust_Dict[str(k)]=Clust_Dict[str(k)]+i
  # print(Clust_Dict[str(k)])
  # print("\n\n\n\n\n\n\n\n\n\n")
  j=j+1
for x,y in Clust_Dict.items():
  print(x,len(y))


NameError: ignored

In [ ]:
#Now after mapping each and every pixel into its respective labels
#spectral angular distance is to be found between each and every pixel and its centre
#after that the minimum distance pixel are considered to be our new endmember